In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

In [27]:
# Parameters
states = ['A', 'B', 'C']
demand = {'A': 5000, 'B': 7000, 'C': 4000}  # MWh
solar_max_capacity = {'A': 5, 'B': 2, 'C': 8}  # MW
gas_existing_capacity = {'A': 1.5, 'B': 2, 'C': 1}  # MW
solar_investment_cost = 300  # $/MW
solar_operating_cost = 10  # $/MWh
gas_operating_cost = 20  # $/MWh
gas_investment_cost = 150  # $/MW
emissions_factor_gas = 0.5  # tons CO2/MWh
emissions_cap = 2000  # tons CO2
hours_per_year = 8760  # hours in a year
transmission_cost = 0  # $/MWh
transmission_capacity = {
    ('A', 'B'): 2000,  # MWh max capacity between A and B
    ('B', 'C'): 2000   # MWh max capacity between B and C
}

# Problem definition
model = LpProblem("SAPP_Energy_Planning", LpMinimize)

# Transmission decision variables
transmission = {
    ('A', 'B'): LpVariable("transmission_AB", lowBound=0, upBound=transmission_capacity[('A', 'B')]),
    ('B', 'A'): LpVariable("transmission_BA", lowBound=0, upBound=transmission_capacity[('A', 'B')]),
    ('B', 'C'): LpVariable("transmission_BC", lowBound=0, upBound=transmission_capacity[('B', 'C')]),
    ('C', 'B'): LpVariable("transmission_CB", lowBound=0, upBound=transmission_capacity[('B', 'C')])
}

# Decision variables
solar_generation = {state: LpVariable(f"solar_generation_{state}", lowBound=0) for state in states}  # MWh
gas_generation = {state: LpVariable(f"gas_generation_{state}", lowBound=0) for state in states}  # MWh
solar_capacity = {state: LpVariable(f"solar_capacity_{state}", lowBound=0, upBound=solar_max_capacity[state]) for state in states}  # MW
new_gas_capacity = {state: LpVariable(f"new_gas_capacity_{state}", lowBound=0) for state in states}  # MW

# Objective function: Minimize total cost
model += lpSum([
    solar_investment_cost * solar_capacity[state] +
    solar_operating_cost * solar_generation[state] +
    gas_operating_cost * gas_generation[state] +
    gas_investment_cost * new_gas_capacity[state]
    for state in states]) + lpSum([
    transmission_cost * transmission[line] for line in transmission])

# Constraints
# Energy demand for each state - The balance constraint for each state to include transmission
model += solar_generation['A'] + gas_generation['A'] + transmission[('B', 'A')] - transmission[('A', 'B')] >= demand['A'], "Balance_A"
model += solar_generation['B'] + gas_generation['B'] + transmission[('A', 'B')] + transmission[('C', 'B')] - (transmission[('B', 'A')] + transmission[('B', 'C')]) >= demand['B'], "Balance_B"
model += solar_generation['C'] + gas_generation['C'] + transmission[('B', 'C')] - transmission[('C', 'B')] >= demand['C'], "Balance_C"


# Solar generation limit by installed capacity
for state in states:
    model += solar_generation[state] <= 1000 * solar_capacity[state], f"SolarCapacityLimit_{state}"

# Gas generation limit by total capacity (existing + new)
for state in states:
    model += gas_generation[state] <= hours_per_year * (gas_existing_capacity[state] + new_gas_capacity[state]), f"GasTotalCapacity_{state}"

# Combined emissions cap
model += lpSum([emissions_factor_gas * gas_generation[state] for state in states]) <= emissions_cap, "EmissionsCap"

# Solve the problem
model.solve()

# Extract updated results including transmission
results_with_transmission = {
    "Demand (MWh)": {state: demand[state] for state in states},
    "Solar Generation (MWh)": {state: solar_generation[state].varValue for state in states},
    "Gas Generation (MWh)": {state: gas_generation[state].varValue for state in states},
    "Solar Capacity Installed (MW)": {state: solar_capacity[state].varValue for state in states},
    "New Gas Capacity Installed (MW)": {state: new_gas_capacity[state].varValue for state in states},
    "Transmission (MWh)": {line: transmission[line].varValue for line in transmission},
    "Total Cost ($)": model.objective.value()
}


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/miniconda3/envs/DCOP/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_z/9k_3hrq16r35m7ch79yggf0m0000gn/T/2a9f89eb37514502b16f42a8328bfc0c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_z/9k_3hrq16r35m7ch79yggf0m0000gn/T/2a9f89eb37514502b16f42a8328bfc0c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 57 RHS
At line 68 BOUNDS
At line 76 ENDATA
Problem MODEL has 10 rows, 16 columns and 29 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-6) rows, 10 (-6) columns and 17 (-12) elements
0  Obj 19997.999 Primal inf 15000.1 (3)
5  Obj 193900
Optimal - objective value 193900
After Postsolve, objective 193900, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 193900 - 5 iterations time 0.002, Presolve 0.00
Option for prin

In [28]:
# Print results
for key, value in results_with_transmission.items():
    print(f"{key}: {value}")


Demand (MWh): {'A': 5000, 'B': 7000, 'C': 4000}
Solar Generation (MWh): {'A': 5000.0, 'B': 2000.0, 'C': 6000.0}
Gas Generation (MWh): {'A': 2000.0, 'B': 1000.0, 'C': 0.0}
Solar Capacity Installed (MW): {'A': 5.0, 'B': 2.0, 'C': 6.0}
New Gas Capacity Installed (MW): {'A': 0.0, 'B': 0.0, 'C': 0.0}
Transmission (MWh): {('A', 'B'): 2000.0, ('B', 'A'): 0.0, ('B', 'C'): 0.0, ('C', 'B'): 2000.0}
Total Cost ($): 193900.0
